In [430]:
import pandas as pd

In [431]:
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')
#event.head()
#purchase.head()

Таблица Event
id	идентификатор события
user_id	уникальный идентификатор пользователя, совершившего событие в приложении
start_time	дата и время события
event_type	тип события (значения: registration — регистрация; tutorial_start — начало обучения; tutorial_finish — завершение обучения; level_choice — выбор уровня сложности; pack_choice — выбор пакетов вопросов)
tutorial_id	идентификатор обучения (этот идентификатор есть только у событий обучения)
selected_level	выбранный уровень сложности обучения

Таблица purchase
id	идентификатор события
user_id	уникальный идентификатор пользователя, совершившего событие в приложении
event_datetime	дата и время события/покупки
amount	сумма оплаты

In [500]:
#events_df.info()
#purchase_df.info()
events_df.head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
5,28908,tutorial_start,NaN,2016-05-12T02:11:29,17563.0,12584
6,28909,tutorial_finish,NaN,2016-05-12T02:14:14,17563.0,12584
7,28910,level_choice,hard,2016-05-12T05:30:25,NaN,12584
8,28911,pack_choice,NaN,2016-05-12T05:38:24,NaN,12584
9,28912,tutorial_start,NaN,2016-05-12T07:19:01,17564.0,12583


Период анализа — с 1 января по 31 декабря 2018 года.

In [434]:
#events_df.start_time = pd.to_datetime(events_df.start_time, format='%Y-%m-%dT%H:%M:%S')
event_2018 = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
#Сформируем список пользователей, зарегистриванных в 2018.
registered = events_df[event_2018]['user_id'].to_list() 
events = events_df[events_df.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')
events.info()

C:\Users\Pavel\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


In [435]:
#events_df[events_df['event_type'] == 'level_choice'].info()
events_df[events_df['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125103 entries, 3 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              125103 non-null  int64  
 1   event_type      125103 non-null  object 
 2   selected_level  0 non-null       object 
 3   start_time      125103 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         125103 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 6.7+ MB


In [436]:
#исследуем значения
events.describe(include='all')

C:\Users\Pavel\AppData\Local\Temp/ipykernel_9104/2273988056.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  events.describe(include='all')


,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66959.000000,66959,8342,66959,32954.000000,66959.000000
unique,NaN,5,3,66809,NaN,NaN
top,NaN,registration,medium,2018-03-13 09:37:43,NaN,NaN
freq,NaN,19926,4645,9,NaN,NaN
first,NaN,NaN,NaN,2018-01-01 03:48:40,NaN,NaN
last,NaN,NaN,NaN,2019-01-01 05:50:36,NaN,NaN
mean,113787.000045,NaN,NaN,NaN,40532.934393,37781.543362
std,19329.542752,NaN,NaN,NaN,5213.486632,5751.497904
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97047.500000,NaN,NaN,NaN,36008.250000,32849.000000


In [437]:
events['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

In [438]:
events['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [439]:
# оценим, какое количество пользователей совершали события:
events['user_id'].nunique()

19926

In [440]:
purchase = purchase_df[purchase_df.user_id.isin(registered)]
purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')
purchase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


C:\Users\Pavel\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [441]:
purchase.describe()

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


посмотрим, сколько пользователей совершают событие registration.

In [442]:
events[events['event_type'] == 'registration']['user_id'].nunique()

19926

In [443]:
events['user_id'].nunique()

19926

Посмотрим на срез данных по событию tutorial_start.

In [444]:
events[events['event_type'] == 'tutorial_start'].head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51412,80315,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836
51413,80316,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835
51415,80318,tutorial_start,NaN,2018-01-01 15:40:43,31507.0,27836
51417,80320,tutorial_start,NaN,2018-01-01 17:47:40,31508.0,27833
51420,80323,tutorial_start,NaN,2018-01-01 19:11:36,31509.0,27839
51422,80325,tutorial_start,NaN,2018-01-01 19:46:11,31510.0,27834
51434,80337,tutorial_start,NaN,2018-01-02 02:07:07,31511.0,27840
51435,80338,tutorial_start,NaN,2018-01-02 03:03:44,31512.0,27845
51437,80340,tutorial_start,NaN,2018-01-02 04:55:11,31513.0,27842
51446,80349,tutorial_start,NaN,2018-01-02 07:08:00,31514.0,27845


Посмотрим на количество пользователей, которые совершают событие tutorial_start:

In [445]:
events[events['event_type'] == 'tutorial_start']['user_id'].nunique()

11858

определим процент пользователей, которые перешли к выполнению обучения, и запишем его в переменную percent_tutorial_start_users.

In [446]:
registered_users_count = events[events["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events[events["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print(
    "Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_tutorial_start_users
    )
)

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


 посмотрим, какое количество пользователей проходит обучение до конца (событие tutorial_finish).

In [447]:
events[events['event_type'] == 'tutorial_finish']['user_id'].nunique()

10250

Рассчитаем процент пользователей, завершивших обучение, среди пользователей, которые начали обучение. 
Это будет показатель tutorial_completion_rate (коэффициент «завершаемости» обучения).

In [448]:
tutorial_finish_users_count = events[events["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print(
    "Процент пользователей, завершивших обучение: {:.2%}".format(
        tutorial_completion_rate
    )
)

Процент пользователей, завершивших обучение: 86.44%


Следующий этап — это выбор уровня сложности level_choice. Давайте посмотрим на процент тех, кто доходит до этого этапа.

In [449]:
events[events['event_type'] == 'level_choice']['user_id'].nunique()

8342

Из логики нашего приложения следует, что выбор уровня сложности можно совершить и без прохождения обучения. 
Поэтому рассчитаем долю пользователей percent_level_choice_users, которые выбрали уровень сложности, от общего числа зарегистрировавшихся:

In [450]:
level_choice_users_count = events[events["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


### Вывод
Меньше половины пользователей (41.76 %) доходят до этапа выбора уровня сложности вопросов. А ведь этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате.

Таким образом, для успешной монетизации приложения крайне важно оптимизировать прохождение до этапа выбора сложности.

определим число пользователей, которые совершили событие выбора бесплатного пакета вопросов pack_choice.

In [451]:
events[events['event_type'] == 'pack_choice']['user_id'].nunique()

5737

 Оценим процент таких пользователей от числа пользователей, которые выбрали уровень сложности.

In [452]:
training_choice_users_count = events[events["event_type"] == "pack_choice"][
    "user_id"
].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count
print(
    "Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_training_choice_users
    )
)

Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): 68.77%


Следующий этап после выбора бесплатного набора вопросов — это возможная покупка платных пакетов. 
Оценим число пользователей, которые совершили покупку. Для этого нам потребуется датафрейм purchase_df.

In [453]:
purchase['user_id'].nunique()

1600

Рассчитаем процент пользователей percent_of_paying_users, которые оплатили вопросы, от числа пользователей, которые приобрели бесплатные вопросы:

In [454]:
paying_users_count = purchase["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): 27.89%


посмотрим, какой процент составляют покупатели от общего числа зарегистрировавшихся. Для этого создадим переменную purchase_rate.

In [455]:
purchase_rate = paying_users_count / registered_users_count
print(
    "Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): {:.2%}".format(
        purchase_rate
    )
)

Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): 8.03%


### Вывод
 процент зарегистрировавшихся пользователей, которые приобрели платный пакет, составляет 8.03 %.

Само по себе это число нам ни о чём не говорит, поэтому в рамках одной из гипотез мы рассмотрим этот процент для пользователей разных групп. И эта информация укажет нам возможные пути оптимизации.

# АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ

объединим датафрейм с событиями с датафреймом по оплатам. Это позволит анализировать все эти события в рамках одной структуры данных.

Добавим в датафрейм purchase_df столбец event_type, который будет содержать одно значение purchase. Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

In [456]:
purchase['event_type'] = 'purchase'

C:\Users\Pavel\AppData\Local\Temp/ipykernel_9104/1009542599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase['event_type'] = 'purchase'


Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их несколько отличается, так как столбец id в events_df указывает на идентификатор события, а столбец id в purchase_df указывает на идентификатор оплаты. 
Поэтому применим функцию rename(), чтобы переименовать столбцы в датафреймах.

In [457]:
events = events.rename(columns={"id": "event_id"})
purchase = purchase.rename(columns={"id": "purchase_id"})

Следующим шагом объединим датафреймы events_df и purchase_df с помощью функции pd.concat() и запишем объединённый датафрейм в переменную total_events_df.

In [458]:
total_events_df = pd.concat([events,purchase],sort=False)
total_events_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
51410,80313.0,registration,NaN,2018-01-01 14:08:40,NaN,27837,NaN,NaT,NaN
51411,80314.0,registration,NaN,2018-01-01 14:42:58,NaN,27838,NaN,NaT,NaN
51412,80315.0,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaN,NaT,NaN
51413,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaT,NaN
51414,80317.0,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaN,NaT,NaN


In [459]:
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() и отсортируем все события по возрастанию времени с помощью sort_values().

In [460]:
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

### Воссоздаём последовательность событий

Теперь можно для каждого пользователя создать список, который будет содержать во временной последовательности все события, совершаемые данным пользователем.

Для этого воспользуемся методом groupby по столбцу event_type и применим агрегирующую функцию apply(list). Таким образом мы сгруппируем строки по пользователю, а затем объединим в списки содержимое столбца event_type по каждому пользователю. Запишем результат в датафрейм user_path_df.

In [461]:
user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


### Выявляем наиболее популярные пути

преобразуем список событий в строку event_path. Эта операция нужна для оптимизации скорости объединения, так как иначе Pandas может делать подсчёт слишком долго.

In [462]:
user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_path_df["event_path"].head()

0                                         registration
1      registration > tutorial_start > tutorial_finish
2      registration > tutorial_start > tutorial_finish
3    registration > tutorial_start > tutorial_finis...
4    registration > tutorial_start > tutorial_start...
Name: event_path, dtype: object

Теперь можно сгруппировать датафрейм по столбцу event_path, подсчитав число пользователей.

In [463]:
user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)

Выведем 10 самых популярных последовательностей.

In [464]:
user_paths.head(10)

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

## Вывод

Как мы видим, среди 10 самых популярных последовательностей только одна содержит этап оплаты. Это последовательность registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase.

### Ищем последовательности с оплатой

In [465]:
user_paths[user_paths.index.str.contains('purchase')].head(10)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                                             1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                                                                                                124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase                                                                           101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                            52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                 

### Вывод
Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения. Это наводит нас на гипотезу, что вероятность оплаты зависит от того, проходил ли пользователь обучение.

## АНАЛИЗ ВРЕМЕННЫХ ПРОМЕЖУТКОВ

#### Определяем время между registration и tutorial_start

Сначала определим, какое время проходит между регистрацией и началом обучения.

Важно понимать, что это время мы можем определить только для таких пользователей, которые хотя бы раз начинали обучение.

registration
Первым делом выделим отдельный датафрейм registration_df, который будет содержать только события с event_type = registration. Этот датафрейм будет вспомогательным для определения времени между регистрацией и началом обучения.

In [466]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration']

Проверим, что в нашем датафрейме registration_df для каждого пользователя содержится только по одному событию, то есть по одной регистрации на пользователя.

Для определения среднего количества событий на пользователя мы можем воспользоваться методом value_counts(), результатом которого будет серия с количеством событий на каждого пользователя. Затем, усреднив эти значения между собой, мы получим среднее число событий типа registration на одного пользователя.

In [467]:
registration_df['user_id'].value_counts().mean()

1.0

In [468]:
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)

tutorial_start
Теперь возьмёмся за следующее событие. Выделим отдельный датафрейм tutorial_start_df, который будет содержать только события с event_type = tutorial_start (начало обучения).

In [469]:
tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']
tutorial_start_df['user_id'].value_counts().mean()

1.522179119581717

больше 1го т.к. Один пользователь может начать более чем одно обучение за время пользования приложением. 

У некоторых наших пользователей более чем по одному событию начала обучения. Оставим только первое обучение. Таким образом мы будем искать время между регистрацией и началом первого обучения пользователя.

In [470]:
tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values(
    "start_time"
).drop_duplicates("user_id")
tutorial_start_df_wo_duplicates['user_id'].value_counts().mean()

1.0

In [471]:
tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[
    ["user_id", "tutorial_id", "start_time"]
].rename(columns={"start_time": "tutorial_start_time"})

Объединить датафреймы мы можем с помощью функции merge(), объединяя данные по параметру user_id. Также нам нужно объединить данные с использованием параметра how='inner', это позволит в объединённом датафрейме оставить только такие идентификаторы пользователей, которые есть в обоих датафреймах. 

In [472]:
merged_df = registration_df.merge(
    tutorial_start_df_wo_duplicates, on="user_id", how="inner"
)
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36


In [473]:
# Сделаем столбец timedelta, в котором посчитаем разницу между временем начала обучения (tutorial_start_time)
# и временем регистрации (registration_time):
# merged_df['tutorial_start_time'] = pd.to_datetime(merged_df['tutorial_start_time'])
# merged_df['registration_time'] = pd.to_datetime(merged_df['registration_time'])
merged_df["timedelta"] = (
    merged_df["tutorial_start_time"] - merged_df["registration_time"]
)
merged_df.head()


,user_id,registration_time,tutorial_id,tutorial_start_time,timedelta
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40,0 days 13:40:15
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11,0 days 11:11:01
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51,0 days 03:06:04
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40,0 days 01:26:33
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36,0 days 00:47:35


In [474]:
merged_df["timedelta"].mean()

Timedelta('0 days 04:38:24.019817844')

In [475]:
merged_df['timedelta'].describe()

count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta, dtype: object

четверть пользователей тратит меньше 1 часа 22 минут на переход от регистрации к началу обучения;
половина всех пользователей тратит между регистрацией и началом обучения менее 3 часов 23 минут.

Мы можем сосредоточится не на оптимизации среднего времени (поскольку среднее будет увеличено за счёт пользователей, которые долго проходят этап), а на том, чтобы оптимизировать медианное время.

### Измеряем время обучения

Сначала сформируем датафрейм tutorial_finish_df, который содержит события окончания обучения.

In [476]:
tutorial_finish_df = total_events_df[total_events_df["event_type"] == "tutorial_finish"]

 возьмём время окончания первого обучения для тех обучений, которые были закончены. Для этого сформируем список с идентификаторами первых обучений. 

In [477]:
# возьмём время окончания первого обучения для тех обучений, которые были закончены. Для этого сформируем список с идентификаторами первых обучений.
first_tutorial_ids = tutorial_start_df_wo_duplicates["tutorial_id"].unique()

In [478]:
#отфильтруем датафрейм tutorial_finish_df, оставив в нём только события для таких обучений, которые были первыми для пользователя.
tutorial_finish_df = tutorial_finish_df[
    tutorial_finish_df["tutorial_id"].isin(first_tutorial_ids)
]
tutorial_finish_df['user_id'].value_counts().mean() #Проверка, что для каждого пользователя содержится по одному событию завершения обучения:

1.0

- Объединим датафреймы двух событий.
- Посчитаем разницу во времени между событиями.
- Выведем характеристики разницы.

In [479]:
tutorial_finish_df = tutorial_finish_df[["user_id", "start_time"]].rename(
    columns={"start_time": "tutorial_finish_time"}
)

merged_df_2 = tutorial_start_df_wo_duplicates.merge(
    tutorial_finish_df, on="user_id", how="inner"
)
#merged_df_2['tutorial_start_time'] = pd.to_datetime(merged_df_2['tutorial_start_time'])
#merged_df_2['tutorial_finish_time'] = pd.to_datetime(merged_df_2['tutorial_finish_time'])

merged_df_2["timedelta"] = (
    merged_df_2["tutorial_finish_time"] - merged_df_2["tutorial_start_time"]
)
merged_df_2.head()
print(merged_df_2["timedelta"].mean())
print(merged_df_2["timedelta"].describe())

0 days 00:03:53.174160732
count                         9830
mean     0 days 00:03:53.174160732
std      0 days 00:01:44.803217992
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%                0 days 00:05:08
max                0 days 00:10:06
Name: timedelta, dtype: object


### Вывод.
Таким образом, 75 % пользователей проходят обучение менее чем за 5 минут 8 секунды

### Определяем время между registration и level_choice

In [480]:
# рассчитаем время между выбором уровня сложности и регистрацией
level_choice_df = total_events_df[total_events_df["event_type"] == "level_choice"]
print(level_choice_df["user_id"].value_counts().mean())
level_choice_df = level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)
merged_df_3 = registration_df.merge(level_choice_df, on="user_id", how="inner")
## пребразуем в тип дата
# merged_df_3['level_choice_time'] = pd.to_datetime(merged_df_3['level_choice_time']) 
# merged_df_3['registration_time'] = pd.to_datetime(merged_df_3['registration_time'])
#считаем timedelta
merged_df_3["timedelta"] = (
    merged_df_3["level_choice_time"] - merged_df_3["registration_time"]
)
print(merged_df_3["timedelta"].mean())
print(merged_df_3["timedelta"].describe())

1.0
0 days 07:10:19.169863342
count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


* (1) Сколько в среднем для всех пользователей проходит времени между событием выбора уровня сложности тренировки до события выбора набора бесплатных вопросов?
* (2) Сколько в среднем для всех пользователей проходит времени между событием выбора бесплатных вопросов и первой оплатой?

* уровень сложности тренировки - level choice (в event_type)
* Выбор набора бесплатный вопросов - pack_choice (в event_type) другое название **Training Choice**
- -Таблица Event

In [481]:
# считаем среднее для всех пол-ей... между уровнем сложности тренировки до выбора набора беспл вопр.
training_choice_df = total_events_df[total_events_df["event_type"] == "pack_choice"]
training_choice_df_wo_duplicates = training_choice_df.sort_values(
    "start_time"
).drop_duplicates("user_id")
#training_choice_df_wo_duplicates['user_id'].value_counts().mean() # проверяем чтобы среднее было "1"
training_choice_df_wo_duplicates = training_choice_df_wo_duplicates[["user_id", "start_time"]].rename(
    columns={"start_time": "pack_choice_time"}
)
merged_df_4 = level_choice_df.merge(
    training_choice_df_wo_duplicates, on="user_id", how="inner"
)
merged_df_4["timedelta"] = (
    merged_df_4["pack_choice_time"] - merged_df_4["level_choice_time"]
)
print(merged_df_4["timedelta"].mean())
print(merged_df_4["timedelta"].describe())

0 days 00:05:17.128464354
count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta, dtype: object


### Вывод (1)
Более 5 минут - столько в среднем для всех пользователей проходит времени между событием выбора уровня сложности до события выбора набора бесплатных вопросов

In [482]:
# считаем среднее между выбором сложности и первой оплатой
first_purchase_df = total_events_df[total_events_df["event_type"] == "purchase"]
first_purchase_df_wo_duplicates = first_purchase_df.sort_values(
    "event_datetime"
).drop_duplicates("user_id")
# first_purchase_df_wo_duplicates['user_id'].value_counts().mean() # проверяем чтобы среднее было "1"
first_purchase_df_wo_duplicates = first_purchase_df_wo_duplicates[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "first_purchase_time"}
)
merged_df_5 = first_purchase_df_wo_duplicates.merge(
    level_choice_df, on="user_id", how="inner"
)
merged_df_5["timedelta"] = (
    merged_df_5["first_purchase_time"] - merged_df_5["level_choice_time"]
)
print(merged_df_5["timedelta"].mean())
print(merged_df_5["timedelta"].describe())

3 days 17:52:17.719375
count                         1600
mean        3 days 17:52:17.719375
std      2 days 04:37:14.656359486
min                0 days 00:49:20
25%         1 days 21:27:33.750000
50%                3 days 12:59:27
75%         5 days 09:47:33.750000
max               10 days 18:35:09
Name: timedelta, dtype: object


### Вывод (2)
Более 3 дней - столько в среднем для всех пользователей проходит времени между событием выбора бесплатных вопросов и первой оплатой

# Проверка аналитической гипотезы

Определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:

* пользователи, которые прошли обучение хотя бы раз;
* пользователи, которые начали обучение, но не прошли его ни разу;
* пользователи, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

### Ищем пользователей, прошедших обучение хотя бы раз

In [483]:
users_with_finished_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_finish"
]["user_id"].unique()
print(len(users_with_finished_tutorial))

10250


### Ищем пользователей, которые начали, но не закончили обучение

In [484]:
# Это будут пользователи, у которых есть событие tutorial_start, но нет события tutorial_finish.
users_with_started_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_start"
]["user_id"].unique()
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = (
    set_users_with_started_tutorial.difference(set(users_with_finished_tutorial))
)

print(len(set_users_with_started_tutorial))
print(len(set_users_not_finished_but_started_tutorial))
print(
    len(set_users_with_started_tutorial) - len(set(users_with_finished_tutorial))
    == len(set_users_not_finished_but_started_tutorial)
)

11858
1608
True


### Ищем пользователей, сразу выбравших уровень сложности

In [485]:
# У таких пользователей отсутствует событие tutorial_start. 
# Поэтому мы можем просто взять и убрать из множества всех пользователей множество set_users_with_started_tutorial:
all_users = total_events_df["user_id"].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(
    set_users_with_started_tutorial
)
print('получилось', len(set_users_not_started_tutorial), 'пользователей, которые ни разу не начинали обучение.')
print(
    len(set_all_users) - len(set_users_with_started_tutorial)
    == len(set_users_not_started_tutorial)
)

получилось 8068 пользователей, которые ни разу не начинали обучение.
True


Проверим, что мы правильно разбили пользователей на группы.

Просуммировав длину всех множеств пользователей, мы должны получить исходное количество всех пользователей:

In [486]:
len(set_users_not_finished_but_started_tutorial) + len(set_users_not_started_tutorial) + len(users_with_finished_tutorial) == len(set_all_users)

True

### Считаем пользователей, которые завершили обучение и совершили оплату

In [487]:
# Сначала сделаем датафрейм purchase_df_1, в котором будут данные по оплатам пользователей, которые завершили обучение.
purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_finished_tutorial)]
purchase_df_1['user_id'].nunique() #  посчитаем, сколько пользователей в этом датафрейме.

1447

Поделив количество пользователей в датафрейме purchase_df_1 на общее количество пользователей, которые завершили обучение, мы получим процент оплативших пользователей в этой группе.

In [488]:
percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / len(
    users_with_finished_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): {:.2%}".format(
        percent_of_purchase_1
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): 14.12%


### Считаем средний размер платежа разных групп

In [489]:
purchase_df_1['amount'].mean()

110.98825155494126

средний размер платежа у пользователей, которые завершили обучение, составляет **110** рублей **98** копеек

сделаем датафрейм purchase_df_2, в котором будут оплаты пользователей, начавших, но не закончивших обучение. И найдём, какой процент таких пользователей оплачивает пакеты вопросов, от общего числа пользователей:

In [490]:
purchase_df_2 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_finished_but_started_tutorial)
]
print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(
    set_users_not_finished_but_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): {:.2%}".format(
        percent_of_purchase_2
    )
)

131
Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): 8.15%


In [491]:
# средний чек
purchase_df_2['amount'].mean()

104.9618320610687

### Определим, как обстоят дела у последней группы — пользователей, которые ни разу не начинали обучение.

* (1) Какой процент пользователей, ни разу не начинавших обучение, совершает оплаты?
* (2) Какова средняя величина оплаты у этих пользователей?

In [492]:
#не начали обучение set_users_not_started_tutorial 
purchase_df_3 = purchase_df[purchase_df["user_id"].isin(set_users_not_started_tutorial)]
percent_of_purchase_3 = purchase_df_3['user_id'].nunique() / len(
    set_users_not_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение): {:.2%}".format(
        percent_of_purchase_3
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение): 0.27%


In [493]:
a = purchase_df_3['amount'].mean()
print('средняя величина оплаты у этих пользователей: {:.5}'.format(a), 'руб')

средняя величина оплаты у этих пользователей: 128.41 руб
